<a href="https://colab.research.google.com/github/jahnavinischal/loksamvad/blob/main/multilingual_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=a3d968e91b1b973457cd7c4847457b438e3b3e7f87115276f8394a6aa69f114c
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [6]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=c821abc838b940d2e8985bd151300d635197180ed84b657df83c7f772c97ffb9
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling ch

In [7]:
!pip install indic-nlp-library


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 14.0 MB/s eta 0:00:00


In [8]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.1 MB/s eta 0:00:00


In [9]:
!pip install indic-nlp-library


In [11]:
import indicnlp

print(indicnlp.__path__)


['/usr/local/lib/python3.10/dist-packages/indicnlp']


In [ ]:
import random
from transformers import AutoModelForCausalLM, AutoTokenizer
import nltk
import string
from indicnlp.tokenize import sentence_tokenize, indic_tokenize
from indicnlp import common



# Load English and Multilingual BERT models and tokenizers
english_model = "bert-base-uncased"
multilingual_model = "bert-base-multilingual-cased"
english_tokenizer = AutoTokenizer.from_pretrained(english_model)
multilingual_tokenizer = AutoTokenizer.from_pretrained(multilingual_model)


def read_question_answers(file_path):
    qa_pairs = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        for i in range(0, len(lines), 2):
            question = lines[i].strip()
            answer = lines[i + 1].strip()
            qa_pairs[question] = answer
    return qa_pairs

english_qa_pairs = read_question_answers('/content/faqsdata.txt')
hindi_qa_pairs = read_question_answers('/content/hindifaq.txt')


# Define greetings and responses for both languages
english_greet_inputs = ('hello', 'hi', 'how are you?', 'what\'s up')
english_greet_responses = ('Hello', 'Hi', "I'm good, how about you?", 'Not much, how can I help you?')

hindi_greet_inputs = ('नमस्ते', 'नमस्कार', 'कैसे हो?', 'क्या चल रहा है?')
hindi_greet_responses = ('नमस्ते', 'नमस्कार', 'मैं ठीक हूँ, आपका क्या हाल है?', 'सब ठीक है, मैं आपकी कैसे सहायता कर सकता हूँ?')

# Define a function to determine the language of user input
def detect_language(user_input):
    if any(word in user_input.lower() for word in english_greet_inputs):
        return 'english'
    elif any(word in user_input for word in hindi_greet_inputs):
        return 'hindi'
    else:
        return 'english'  # Default to English if no language is detected

def generate_response(user_input, language):
    user_input = user_input.lower()

    # Preprocess user input to remove punctuation and extra spaces
    user_input = ''.join([char for char in user_input if char not in string.punctuation])
    user_input = ' '.join(user_input.split())  # Remove extra spaces

    if language == 'english':
        if user_input in english_greet_inputs:
            return random.choice(english_greet_responses)
        elif user_input in english_qa_pairs:
            return english_qa_pairs[user_input]
        else:
            return "I'm sorry, I didn't understand that."

    elif language == 'hindi':
        if any(word in user_input for word in hindi_greet_inputs):
            return random.choice(hindi_greet_responses)
        elif user_input in hindi_qa_pairs:
            return hindi_qa_pairs[user_input]
        else:
            return "मुझे खेद है, मुझे समझ नहीं आया।"

#

# Implement the main conversation loop
print("Hello! I am GARA. I am here to assist you. To end the conversation, type 'bye' or 'exit'.")
while True:
    user_input = input()
    if user_input.lower() in ('bye', 'exit'):
        print("GARA: Goodbye!")
        break
    else:
        language = detect_language(user_input)
        response = generate_response(user_input, language)
        print("GARA:", response)

def generate_response(user_input, language):
    user_input = user_input.lower()
    user_input = ''.join([char for char in user_input if char not in string.punctuation])  # Remove punctuation

    print("User Input:", user_input)
    print("Detected Language:", language)
    print("English Questions:", english_qa_pairs)
    print("Hindi Questions:", hindi_qa_pairs)

    if language == 'english':
        if user_input in english_greet_inputs:
            return random.choice(english_greet_responses)
        elif user_input in english_qa_pairs:
            return english_qa_pairs[user_input]
        else:
            return "I'm sorry, I didn't understand that."

    elif language == 'hindi':
        if any(word in user_input for word in hindi_greet_inputs):
            return random.choice(hindi_greet_responses)
        elif user_input in hindi_qa_pairs:
            return hindi_qa_pairs[user_input]
        else:
            return "मुझे खेद है, मुझे समझ नहीं आया।"



Hello! I am GARA. I am here to assist you. To end the conversation, type 'bye' or 'exit'.
hi
GARA: Not much, how can I help you?
नमस्ते
GARA: नमस्ते
